In [4]:
!pip install -e .
!pip install spacy
!python3 -m spacy download en_core_web_lg
!pip install pandas
!pip install openpyxl


Obtaining file:///Users/rajashreedahal/Desktop/Metadata_Finetuning/larch
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done


  Building editable for larch (pyproject.toml) ... done
  Created wheel for larch: filename=larch-0.0.2a0-0.editable-py3-none-any.whl size=14420 sha256=8654cee9105820fd3405570201e026b260b5f055fb18cf879113365ef9beb317
  Stored in directory: /private/var/folders/m4/9_ydsvdn3b3d84rntrrsxm0h0000gn/T/pip-ephem-wheel-cache-bp9ptwer/wheels/3a/98/39/46f8f1efc02842106044493327f82ab1f2c43a0a7f389a1df9
Successfully built larch
  Attempting uninstall: larch
    Found existing installation: larch 0.0.2a0
    Uninstalling larch-0.0.2a0:
      Successfully uninstalled larch-0.0.2a0

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 13.6 MB/s eta 0:00:00m eta 0:00:010:00:01

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [65]:
import pandas as pd
from larch.metadata import InstructorBasedOpenAIMetadataExtractor
from larch.metadata.validators import WhitelistBasedMetadataValidator
from larch.processors import PIIRemover, TextProcessingPipeline
from larch.utils import load_whitelist

text_processor = TextProcessingPipeline(
    lambda x: re.sub(r"\$(?=\w|\n|\()", " ", x).strip(),
    lambda x: re.sub(r"\)(?=\w|\n|\()", " ", x).strip(),
    lambda x: re.sub(r"\#(?=\w|\n|\()", " ", x).strip(),
    lambda x: x.replace("\t", " ").replace("!", " ").strip(),
    PIIRemover()
)
import re
import requests
from bs4 import BeautifulSoup

from pydantic import BaseModel,constr,Field
from typing import Literal

class Environmental_Justice(BaseModel):
    Indicators: Literal["Disasters","Human Dimensions","Food Availability","Health & Air Quality","Water Availability", "Extreme Heat","Urban Flooding","Climate"]
    Description: str
    Geographic_Coverage: str =Field(pattern=r'')
    Format: str
    Spatial_Resolution: str = Field(pattern=r'^(\d+(\.\d+)? [a-zA-Z]+|varies|N/A)$')
    Temporal_Resolution: str = Field(pattern=r'^(\d+(\.\d+)? [a-zA-Z]+|N/A|varies|weekly|monthly|daily|yearly|varies-multiple datasets included|Daily < Weekly|Hourly < Daily|Weekly < Monthly|Monthly < Yearly|1 minute)$')
    Temporal_Extent: str = Field(pattern=r'^(\d{4}-\d{2}-\d{2} to present|present|\d{4}-\d{2}-\d{2} (?:to|until) present|\d{4}-\d{2}-\d{2} to \d{4}-\d{2}-\d{2}|varies(?:- multiple datasets (?:included|available))?)$')
    Latency: str = Field(pattern=r'^\d+(\s*(Day|days|months|NRT))?$')
    Project: str
    Data_Visualization: bool

    

    
schema = Environmental_Justice
sheet_names=["Water Availability - Cleaned", "Disasters (Disaster Recovery) -", 
 "Food Availability - Cleaned", "Human Dimensions - Cleaned", "Urban Flooding - Cleaned",
 "Extreme Heat - CIP","Health and Air Quality - CIP", "Climate (Climate Change) - CIP"]
sheet_names=["Water Availability - Cleaned"]
mapper = pd.read_excel("nasa_esds.xlsx", sheet_name=sheet_names)
res = {}
url_lists=[]
for field_key, value_df in mapper.items():
    res[field_key] = {}
    cols = value_df.columns
    for i, row in value_df.iterrows():
        source_link_col = [col for col in cols if col == "Source/Link"]
        alternate_vals = filter(None, row[source_link_col])
        alternate_vals=list(alternate_vals)
        if alternate_vals and str(alternate_vals[0]).startswith("http"):
            url_lists.append(alternate_vals[0])

url_content=[]
urls=[]
for url in url_lists:
    response=requests.get(url)
    html_page = response.text
    soup = BeautifulSoup(html_page, "html.parser")
    text=soup.get_text()
    text = re.sub(r'[\t\n\r\f\v]+', '', text)
    text=re.sub(r'[^\w\s]', '', text)
    url_content.append(text)
    urls.append(url)



In [66]:
urls

['https://doi.org/10.5067/MEaSUREs/GFSAD/GFSAD30AFCE.001',
 'https://doi.org/10.5067/MEaSUREs/GFSAD/GFSAD30AUNZCNMOCE.001',
 'https://doi.org/10.5067/MEaSUREs/GFSAD/GFSAD30SEACE.001',
 'https://doi.org/10.5067/MEaSUREs/GFSAD/GFSAD30EUCEARUMECE.001',
 'https://doi.org/10.5067/MEaSUREs/GFSAD/GFSAD30SACE.001',
 'https://fews.net/',
 'https://ldas.gsfc.nasa.gov/fldas',
 'https://dx.doi.org/10.5067/TEMSC-3JC63',
 'https://doi.org/10.7927/H4HH6H08 ',
 'https://doi.org/10.7927/H4N877QK ']

In [67]:

import json

metadata_extractor = InstructorBasedOpenAIMetadataExtractor(
    model="gpt-3.5-turbo-0125",
#     model="gpt-4",
    schema=schema,
    preprocessor=text_processor,
    debug=True,
)

response_result={}
for enum,text in enumerate(url_content):
    metadata = metadata_extractor(text)
    response=json.loads((metadata['choices'][0]['message']["function_call"]["arguments"]))
    response_result[urls[enum]]=response


2024-02-07 12:05:55.431 | DEBUG    | larch.metadata.extractors_openai:_extract:73 - nchars=8105
Text :: LP DAAC  GFSAD30AFCEHomeAboutAbout LP DAAC                                                                                                    News Archive                                                                                                    DataGet Started with Data                                                                                                    Search Data Catalog                                                                                                    Data Citations and Policies                                                                                                    View All DataToolsAppEEARSData PoolData Prep ScriptsNASA Earthdata SearchUSGS EarthExplorerView All ToolsResourcesData in Action                                                                                                    ELearning                                  

the system prompt is Extract metadata and entities details accurately from my requests.
The messages is [{'role': 'system', 'content': 'Extract metadata and entities details accurately from my requests.'}, {'role': 'user', 'content': 'LP DAAC  GFSAD30AFCEHomeAboutAbout LP DAAC                                                                                                    News Archive                                                                                                    DataGet Started with Data                                                                                                    Search Data Catalog                                                                                                    Data Citations and Policies                                                                                                    View All DataToolsAppEEARSData PoolData Prep ScriptsNASA Earthdata SearchUSGS EarthExplorerView All ToolsResourcesData in Action            

2024-02-07 12:05:58.301 | DEBUG    | larch.metadata.extractors_openai:_extract:73 - nchars=8294
Text :: LP DAAC  GFSAD30AUNZCNMOCEHomeAboutAbout LP DAAC                                                                                                    News Archive                                                                                                    DataGet Started with Data                                                                                                    Search Data Catalog                                                                                                    Data Citations and Policies                                                                                                    View All DataToolsAppEEARSData PoolData Prep ScriptsNASA Earthdata SearchUSGS EarthExplorerView All ToolsResourcesData in Action                                                                                                    ELearning                            

the system prompt is Extract metadata and entities details accurately from my requests.
The messages is [{'role': 'system', 'content': 'Extract metadata and entities details accurately from my requests.'}, {'role': 'user', 'content': 'LP DAAC  GFSAD30AUNZCNMOCEHomeAboutAbout LP DAAC                                                                                                    News Archive                                                                                                    DataGet Started with Data                                                                                                    Search Data Catalog                                                                                                    Data Citations and Policies                                                                                                    View All DataToolsAppEEARSData PoolData Prep ScriptsNASA Earthdata SearchUSGS EarthExplorerView All ToolsResourcesData in Action      

2024-02-07 12:06:01.019 | DEBUG    | larch.metadata.extractors_openai:_extract:73 - nchars=8162
Text :: LP DAAC  GFSAD30SEACEHomeAboutAbout LP DAAC                                                                                                    News Archive                                                                                                    DataGet Started with Data                                                                                                    Search Data Catalog                                                                                                    Data Citations and Policies                                                                                                    View All DataToolsAppEEARSData PoolData Prep ScriptsNASA Earthdata SearchUSGS EarthExplorerView All ToolsResourcesData in Action                                                                                                    ELearning                                 

...

the system prompt is Extract metadata and entities details accurately from my requests.
The messages is [{'role': 'system', 'content': 'Extract metadata and entities details accurately from my requests.'}, {'role': 'user', 'content': 'JPL GRACE and GRACEFO Mascon Ocean Ice and Hydrology Equivalent Water Height Coastal Resolution Improvement CRI Filtered Release 061 Version 03  PODAAC  JPL  NASA Search Type WebsiteDataSearch   Jet Propulsion LaboratoryCalifornia Institute of Technology PODAACFollow UsNavigation   Search Type WebsiteDataSearch HomeFind DataScience DisciplinesOcean Cryosphere Terrestrial Hydrosphere MeasurementsGlaciersIce SheetsGravityGravitational FieldOcean CirculationOcean Heat BudgetOcean Surface TopographyOcean TemperatureOcean WavesOcean WindsSalinityDensitySea IceSurface WaterMissionsADEOSIIAQUAAQUARIUSSACDCOWVRTEMPESTCYGNSSECCOGEOS3GHRSSTGRACEGRACEFOISSRAPIDSCAT JASON 1JASON 3MEASURESCCMPMEASURESMURMEASURESPRESWOTNASASSHNSCATOMGOPERAOSTMJASON 2QUIKSCATSMODESNPPSA

2024-02-07 12:06:13.474 | DEBUG    | larch.metadata.extractors_openai:_extract:73 - nchars=3942
Text :: Reservoirs v101 Global Reservoir and Dam GRanD v1  SEDAC Socioeconomic Data and Applications Center sedacA Data Center in NASAs Earth Observing System Data and Information System EOSDIS  Hosted by CIESIN at Columbia UniversitySearch SEDACDataData SetsMapsWeb Pagessearch Data Data Sets Data Collections Featured Data Uses Data Citations Citations Database Data SubmissionMaps Map Gallery Map Viewer Map Services Mapping ToolsThemes Agriculture Climate Conservation Governance Hazards Health Infrastructure Land Use Marine and Coastal Population Poverty Remote Sensing Sustainability Urban WaterResources Guides Multimedia Networks News Publications Related Sites Remote Sensing ToolsSocial Media Twitter FaceBook YouTube Flickr Blog Posts CommunitiesAbout About SEDAC User Working Group Privacy User RegistrationHelp Global Reservoir and Dam GRanD v1  Follow Us TwitterFollow Us on FacebookYouTub

the system prompt is Extract metadata and entities details accurately from my requests.
The messages is [{'role': 'system', 'content': 'Extract metadata and entities details accurately from my requests.'}, {'role': 'user', 'content': 'Reservoirs v101 Global Reservoir and Dam GRanD v1  SEDAC Socioeconomic Data and Applications Center sedacA Data Center in NASAs Earth Observing System Data and Information System EOSDIS  Hosted by CIESIN at Columbia UniversitySearch SEDACDataData SetsMapsWeb Pagessearch Data Data Sets Data Collections Featured Data Uses Data Citations Citations Database Data SubmissionMaps Map Gallery Map Viewer Map Services Mapping ToolsThemes Agriculture Climate Conservation Governance Hazards Health Infrastructure Land Use Marine and Coastal Population Poverty Remote Sensing Sustainability Urban WaterResources Guides Multimedia Networks News Publications Related Sites Remote Sensing ToolsSocial Media Twitter FaceBook YouTube Flickr Blog Posts CommunitiesAbout About SED

2024-02-07 12:06:15.419 | DEBUG    | larch.metadata.extractors_openai:_extract:73 - nchars=3877
Text :: Dams v101 Global Reservoir and Dam GRanD v1  SEDAC Socioeconomic Data and Applications Center sedacA Data Center in NASAs Earth Observing System Data and Information System EOSDIS  Hosted by CIESIN at Columbia UniversitySearch SEDACDataData SetsMapsWeb Pagessearch Data Data Sets Data Collections Featured Data Uses Data Citations Citations Database Data SubmissionMaps Map Gallery Map Viewer Map Services Mapping ToolsThemes Agriculture Climate Conservation Governance Hazards Health Infrastructure Land Use Marine and Coastal Population Poverty Remote Sensing Sustainability Urban WaterResources Guides Multimedia Networks News Publications Related Sites Remote Sensing ToolsSocial Media Twitter FaceBook YouTube Flickr Blog Posts CommunitiesAbout About SEDAC User Working Group Privacy User RegistrationHelp Global Reservoir and Dam GRanD v1  Follow Us TwitterFollow Us on FacebookYouTubeFlick

the system prompt is Extract metadata and entities details accurately from my requests.
The messages is [{'role': 'system', 'content': 'Extract metadata and entities details accurately from my requests.'}, {'role': 'user', 'content': 'Dams v101 Global Reservoir and Dam GRanD v1  SEDAC Socioeconomic Data and Applications Center sedacA Data Center in NASAs Earth Observing System Data and Information System EOSDIS  Hosted by CIESIN at Columbia UniversitySearch SEDACDataData SetsMapsWeb Pagessearch Data Data Sets Data Collections Featured Data Uses Data Citations Citations Database Data SubmissionMaps Map Gallery Map Viewer Map Services Mapping ToolsThemes Agriculture Climate Conservation Governance Hazards Health Infrastructure Land Use Marine and Coastal Population Poverty Remote Sensing Sustainability Urban WaterResources Guides Multimedia Networks News Publications Related Sites Remote Sensing ToolsSocial Media Twitter FaceBook YouTube Flickr Blog Posts CommunitiesAbout About SEDAC Use

In [62]:
import json
response=json.loads((metadata['choices'][0]['message']["function_call"]["arguments"]))
print(response)

{'Indicators': 'Poverty', 'Description': 'Global Gridded Relative Deprivation Index (GRDI) v1 is a dataset that characterizes the relative levels of multidimensional deprivation and poverty in each 30 arcsecond (1 km) pixel. It provides a value from 0 to 100, where 100 represents the highest level of deprivation and 0 the lowest.', 'Geographic_Coverage': 'Global', 'Format': 'Raster map', 'Spatial_Resolution': '1 km', 'Temporal_Resolution': 'N/A', 'Temporal_Extent': '2010-2020', 'Latency': 'N/A', 'Project': 'NASA Socioeconomic Data and Applications Center (SEDAC) at Columbia University', 'Data_Visualization': True}


In [68]:
response_result

{'https://doi.org/10.5067/MEaSUREs/GFSAD/GFSAD30AFCE.001': {'Indicators': 'Food Availability',
  'Description': 'The NASA Making Earth System Data Records for Use in Research Environments (MEaSUREs) Global Food Security-support Analysis Data (GFSAD) data product provides cropland extent data over the continent of Africa for the nominal year 2015 at 30-meter resolution. The monitoring of global cropland extent is critical for policymaking and provides important baseline data used in agricultural cropland studies pertaining to water sustainability and food security.',
  'Geographic_Coverage': 'Africa',
  'Format': 'GeoTIFF',
  'Spatial_Resolution': '30 m',
  'Temporal_Resolution': 'Multi-Year',
  'Temporal_Extent': '2013-01-01 to 2016-06-30',
  'Latency': 'N/A',
  'Project': 'MEaSUREs GFSAD',
  'Data_Visualization': False},
 'https://doi.org/10.5067/MEaSUREs/GFSAD/GFSAD30AUNZCNMOCE.001': {'Indicators': 'Food Availability',
  'Description': 'The NASA Making Earth System Data Records for U

In [69]:
with open("extractions/gpt3.5/Water Availability - Cleaned_gpt3_5.json",'w') as file:
    json.dump(response_result,file,indent=4)

In [5]:
import math
import pandas as pd
import json

# Evalutation metrics for sample data sheet

In [16]:

sheet_names=["Water Availability - Cleaned"]
mapper = pd.read_excel("nasa_esds.xlsx", sheet_name=sheet_names)
true_data = {}
for field_key, value_df in mapper.items():
#     res[field_key] = {}
    cols = value_df.columns
    for i, row in value_df.iterrows():
        source_link_col = ["Source/Link","Indicators                     (Select from drop-down list)","Description","Description Simplified","Geographic Coverage","Format","Spatial Resolution","Spatial Resolution (standard)","Temporal Resolution","Temporal Extent","Latency","Project","Data Visualization"]
        alternate_vals=row[source_link_col]
        alternate_vals=list(alternate_vals)
        source_link_col[1]="Indicators"
        alternate_values=alternate_vals[1:]
        value={}
        for enum,i in enumerate(source_link_col[1:]):
            if i=="Data Visualization":
                if isinstance(alternate_values[enum],str):
                    value[i]=True
                else:
                    value[i]=False
            else:
                value[i]=alternate_values[enum]
        true_data[alternate_vals[0]]=value
                
            
        



Index(['In SDE?', 'Unnamed: 1', 'Notes', 'Dataset',
       'Indicators                     (Select from drop-down list)',
       'Description', 'Description Simplified', 'Geographic Coverage',
       'Format', 'Spatial Resolution', 'Spatial Resolution (standard)',
       'Temporal Resolution', 'Temporal Extent', 'Latency', 'Source/Link',
       'Project', 'Strengths', 'Limitations', 'Data Visualization',
       'Intended Use'],
      dtype='object')


In [119]:
true_data

{'https://doi.org/10.5067/MEaSUREs/GFSAD/GFSAD30AFCE.001': {'Indicators': 'Food Availability',
  'Description Simplified': 'The Global Food Security-support Analysis Data (GFSAD) Cropland Extent 2015 Africa 30 m V001 dataset provides cropland extent data over Africa for the year 2015. These data may be useful in agricultural cropland studies related to water sustainability and food security.',
  'Geographic Coverage': 'Africa',
  'Format': 'GeoTIFF',
  'Spatial Resolution': '30m',
  'Spatial Resolution (standard)': '30 meters',
  'Temporal Resolution': nan,
  'Temporal Extent': '2013-01-01 to 2016-06-30',
  'Latency': nan,
  'Project': 'MEaSUREs - Making Earth System Data Records for Use in Research Environments',
  'Data Visualization': False},
 'https://doi.org/10.5067/MEaSUREs/GFSAD/GFSAD30AUNZCNMOCE.001': {'Indicators': 'Food Availability',
  'Description Simplified': 'The Global Food Security-support Analysis Data (GFSAD) Cropland Extent 2015 Australia, New Zealand, China, Mongoli

In [37]:
import json
with open("extractions/gpt3.5/Water Availability - Cleaned_gpt3_5.json",'r') as file:
    pred_data=json.load(file)


In [38]:
print(true_data.keys(),pred_data.keys())

dict_keys(['https://doi.org/10.5067/MEaSUREs/GFSAD/GFSAD30AFCE.001', 'https://doi.org/10.5067/MEaSUREs/GFSAD/GFSAD30AUNZCNMOCE.001', 'https://doi.org/10.5067/MEaSUREs/GFSAD/GFSAD30SEACE.001', 'https://doi.org/10.5067/MEaSUREs/GFSAD/GFSAD30EUCEARUMECE.001', 'https://doi.org/10.5067/MEaSUREs/GFSAD/GFSAD30SACE.001', 'https://fews.net/', 'https://ldas.gsfc.nasa.gov/fldas', 'https://dx.doi.org/10.5067/TEMSC-3JC63', 'https://doi.org/10.7927/H4HH6H08 ', 'https://doi.org/10.7927/H4N877QK ']) dict_keys(['https://doi.org/10.5067/MEaSUREs/GFSAD/GFSAD30AFCE.001', 'https://doi.org/10.5067/MEaSUREs/GFSAD/GFSAD30AUNZCNMOCE.001', 'https://doi.org/10.5067/MEaSUREs/GFSAD/GFSAD30SEACE.001', 'https://doi.org/10.5067/MEaSUREs/GFSAD/GFSAD30EUCEARUMECE.001', 'https://doi.org/10.5067/MEaSUREs/GFSAD/GFSAD30SACE.001', 'https://fews.net/', 'https://ldas.gsfc.nasa.gov/fldas', 'https://dx.doi.org/10.5067/TEMSC-3JC63', 'https://doi.org/10.7927/H4HH6H08 ', 'https://doi.org/10.7927/H4N877QK '])


In [39]:
for key in true_data.keys():
    true_source=true_data[key]
    pred_source=pred_data[key]

In [40]:
pred_source

{'Indicators': 'Water Availability',
 'Description': 'Global Reservoir and Dam Database Version 1 (GRanD v1) contains 6862 records of reservoirs and their associated dams with a cumulative storage capacity of 6197 cubic km. The dams were geospatially referenced and assigned to polygons depicting reservoir outlines at high spatial resolution. Dams have multiple attributes such as name of the dam and impounded river, primary use, nearest city, height, area and volume of reservoir, and year of construction or commissioning.',
 'Geographic_Coverage': 'Global',
 'Format': 'vector map',
 'Spatial_Resolution': 'varies',
 'Temporal_Resolution': 'N/A',
 'Temporal_Extent': '2011',
 'Latency': 'N/A',
 'Project': 'Global Reservoir and Dam Database Version 1 (GRanD v1)',
 'Data_Visualization': True}

In [19]:
true_source

{'Indicators': 'Water Availability',
 'Description': 'The Global Reservoir and Dam Database, Version 1, Revision 01 (v1.01) contains 6,862 records of reservoirs and their associated dams with a cumulative storage capacity of 6,197 cubic km. The dams were geospatially referenced and assigned to polygons depicting reservoir outlines at high spatial resolution. Dams have multiple attributes, such as name of the dam and impounded river, primary use, nearest city, height, area and volume of reservoir, and year of construction (or commissioning). While the main focus was to include all dams associated with reservoirs that have a storage capacity of more than 0.1 cubic kilometers, many smaller dams and reservoirs were added where data were available. The data were compiled by Lehner et al. (2011) and are distributed by the Global Water System Project (GWSP) and by the Columbia University Center for International Earth Science Information Network (CIESIN). For details please refer to the Techn

In [46]:
pred_source

{'Indicators': 'Water Availability',
 'Description': 'Global Reservoir and Dam Database Version 1 (GRanD v1) contains 6862 records of reservoirs and their associated dams with a cumulative storage capacity of 6197 cubic km. The dams were geospatially referenced and assigned to polygons depicting reservoir outlines at high spatial resolution. Dams have multiple attributes such as name of the dam and impounded river, primary use, nearest city, height, area and volume of reservoir, and year of construction or commissioning.',
 'Geographic_Coverage': 'Global',
 'Format': 'vector map',
 'Spatial_Resolution': 'varies',
 'Temporal_Resolution': 'N/A',
 'Temporal_Extent': '2011',
 'Latency': nan,
 'Project': 'Global Reservoir and Dam Database Version 1 (GRanD v1)',
 'Data_Visualization': True}

In [45]:
import numpy as np
pred_source["Latency"] = np.nan if pred_source["Latency"]=="N/A" else pred_source["Latency"]
np.isnan(pred_source["Latency"])==np.isnan(true_source["Latency"])

True

In [172]:
# !pip install sentence_transformers
from difflib import SequenceMatcher
import re 
import torch
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim


encoder = SentenceTransformer('all-mpnet-base-v2').to('cpu')

# Evaluation Metric: Exact Match  #Indicators, temporal resolution, temporal extent,
def evaluate_exact_match(predicted, ground_truth):
    return int(predicted.lower() == ground_truth.lower())

# Evaluation Metric: description, project, geographic measure
def evaluate_similarity_or_edit_distance(predicted, ground_truth1,ground_truth2,encoder):
    
    with torch.no_grad():
        pred_embeddings = encoder.encode(predicted, show_progress_bar=False)
        truth1_embeddings = encoder.encode(ground_truth1, show_progress_bar=False)
        truth2_embeddings = encoder.encode(ground_truth2, show_progress_bar=False)
    similarity1 = cos_sim(pred_embeddings, truth1_embeddings).numpy()[0][0]
    similarity2 = cos_sim(pred_embeddings, truth2_embeddings).numpy() [0][0]
    similarity=max(similarity1,similarity2)
    if similarity>0.7:
        return 1
    else:
        return 0
    
def evaluate_project(predicted, ground_truth,encoder):
    if type(predicted)==float and type(ground_truth)==float:
        return 1
    elif type(ground_truth)==float:
        return 0
    ground_truth=ground_truth.split("-",1)
    with torch.no_grad():
        pred_embeddings = encoder.encode(predicted, show_progress_bar=False)
        truth1_embeddings = encoder.encode(ground_truth[0], show_progress_bar=False)
        similarity = cos_sim(pred_embeddings, truth1_embeddings).numpy()[0][0]
        if len(ground_truth)>1:
            truth2_embeddings = encoder.encode(ground_truth[1], show_progress_bar=False)
            similarity1 = cos_sim(pred_embeddings, truth2_embeddings).numpy()[0][0]
            similarity=max(similarity,similarity1)
    print("prediction",predicted)
    print("ground_truth",ground_truth)
    print("Similarity")
    if similarity>0.58:
        return 1
    else:
        return 0


# Evaluation Metric: Exact Match or at least one match if multiple
def evaluate_format(predicted, ground_truth):
    predicted = np.nan if predicted=="N/A" else predicted
    if type(predicted)==float and type(ground_truth)==float:
        return 1
    predicted=predicted.lower()
    ground_truth=ground_truth.lower()
    if predicted==ground_truth:
        return 1
    predicted_formats = set(predicted.split(","))
    ground_truth_formats = set(ground_truth.split(","))
    if predicted_formats.intersection(ground_truth_formats):
        return 1
    return 0

def evaluate_spatial_resolution(predicted, ground_truth1,ground_truth2):
    predicted = np.nan if predicted=="N/A" else predicted
    if (type(predicted)==float and type(ground_truth1)==float) or (type(predicted)==float and type(ground_truth2)==float):
        return 1
    elif (type(ground_truth1)==float or type(ground_truth2)==float):
        return 0
    elif type(predicted)==float:
        return 0
    predicted=predicted.lower()
    ground_truth1=ground_truth1.lower()
    ground_truth2=ground_truth2.lower()
    if (predicted==ground_truth1) or (predicted==ground_truth2):
        return 1
    predicted = re.sub(r'[a-zA-Z\(\)\[\]]', '', predicted).strip()
    ground_truth1=re.sub(r'[a-zA-Z\(\)\[\]]', '', ground_truth1).strip()
    ground_truth2=re.sub(r'[a-zA-Z\(\)\[\]]', '', ground_truth2).strip()
    if not (ground_truth1 or ground_truth2) and predicted.find(',') != -1:
        return 1
    if predicted==ground_truth1 or predicted==ground_truth2:
        return 1
    if (int(predicted)==int(ground_truth1)) or (int(predicted)==int(ground_truth2)):
        return 1
    return 0


def evaluate_latency(predicted, ground_truth):
    predicted = np.nan if predicted=="N/A" else predicted
    if type(predicted)==float and type(ground_truth)==float:
        return 1
    if type(predicted)==float or type(ground_truth)==float:
        return 0
    predicted=predicted.lower()
    ground_truth=ground_truth.lower()
    if (predicted==ground_truth):
        return 1
    predicted = re.sub(r'[a-zA-Z]', '', predicted)
    ground_truth=re.sub(r'[a-zA-Z]', '', ground_truth)
    if (int(predicted)==int(ground_truth)):
        return 1
    return 0


def evaluate_temporal_resolution_extent(predicted,ground_truth):
    predicted = np.nan if predicted=="N/A" else predicted
    ground_truth=np.nan if (type(ground_truth)==str and ground_truth.lower()=="nan") else ground_truth
    if type(predicted)==float and type(ground_truth)==float:
        return 1
    if type(predicted)==float or type(ground_truth)==float:
        return 0
    predicted=predicted.lower()
    ground_truth=ground_truth.lower()
    if predicted==ground_truth:
        return 1
    predicted = ''.join(re.findall(r'\w', predicted))
    ground_truth=''.join(re.findall(r'\w', ground_truth))
    if predicted==ground_truth:
        return 1 
    return 0

def evaluate_data_visualization_indicators(predicted,ground_truth):
    if predicted==ground_truth:
        return 1
    return 0
    


In [173]:
metrics={}
for key in true_data.keys():
    true_source=true_data[key]
    pred_source=pred_data[key]
    for each_key in pred_source.keys():
        if each_key=="Indicators":
            result=evaluate_exact_match(pred_source[each_key],true_source[each_key])
            if each_key not in metrics.keys():
                    metrics[each_key]=[result]
            else:
                metrics[each_key].append(result)
        elif each_key=="Description":
            result=evaluate_similarity_or_edit_distance(pred_source[each_key], true_source[each_key],true_source["Description Simplified"],encoder)
            if each_key not in metrics.keys():
                metrics[each_key]=[result]
            else:
                metrics[each_key].append(result)
        elif each_key=="Geographic_Coverage":
            evaluate_format(pred_source[each_key],true_source["Geographic Coverage"])
            if each_key not in metrics.keys():
                metrics[each_key]=[result]
            else:
                metrics[each_key].append(result) 
        elif each_key=="Format":
            result=evaluate_format(pred_source[each_key],true_source[each_key])
            if each_key not in metrics.keys():
                metrics[each_key]=[result]
            else:
                metrics[each_key].append(result)
        elif each_key=="Spatial_Resolution":
            result=evaluate_spatial_resolution(pred_source[each_key],true_source["Spatial Resolution"],true_source["Spatial Resolution (standard)"])
            if each_key not in metrics.keys():
                metrics[each_key]=[result]
            else:
                metrics[each_key].append(result)
        elif each_key=="Temporal_Resolution":
            result=evaluate_temporal_resolution_extent(pred_source[each_key],true_source["Temporal Resolution"])
            if each_key not in metrics.keys():
                metrics[each_key]=[result]
            else:
                metrics[each_key].append(result)
        elif each_key=="Temporal_Extent":
            result=evaluate_temporal_resolution_extent(pred_source[each_key],true_source["Temporal Extent"])
            if each_key not in metrics.keys():
                metrics[each_key]=[result]
            else:
                metrics[each_key].append(result)
        elif each_key=="Latency":
            result=evaluate_latency(pred_source[each_key],true_source["Latency"])
            if each_key not in metrics.keys():
                metrics[each_key]=[result]
            else:
                metrics[each_key].append(result)
        elif each_key=="Project":
            result=evaluate_project(pred_source[each_key], true_source[each_key],encoder)
            if each_key not in metrics.keys():
                metrics[each_key]=[result]
            else:
                metrics[each_key].append(result)
        elif each_key=="Data_Visualization":
            result=evaluate_data_visualization_indicators(pred_source[each_key], true_source["Data Visualization"])
            if each_key not in metrics.keys():
                metrics[each_key]=[result]
            else:
                metrics[each_key].append(result)









prediction MEaSUREs GFSAD
ground_truth ['MEaSUREs ', ' Making Earth System Data Records for Use in Research Environments']
prediction MEaSUREs GFSAD
ground_truth ['MEaSUREs ', ' Making Earth System Data Records for Use in Research Environments']
prediction MEaSUREs GFSAD
ground_truth ['MEaSUREs ', ' Making Earth System Data Records for Use in Research Environments']
prediction MEaSUREs GFSAD
ground_truth ['MEaSUREs ', ' Making Earth System Data Records for Use in Research Environments']
prediction MEaSUREs GFSAD
ground_truth ['MEaSUREs ', ' Making Earth System Data Records for Use in Research Environments']
prediction FLDAS (Famine Early Warning Systems Network Land Data Assimilation System)
ground_truth ['FEWS NET ', ' FEWS NET Land Data Assimilation System']
prediction Gravity Recovery and Climate Experiment (GRACE) and Gravity Recovery and Climate Experiment Follow-On (GRACEFO)
ground_truth ['GRACE ', ' Gravity Recovery and Climate Experiment, GRACE', 'FO ', ' Gravity Recovery and C

In [171]:
metrics

{'Indicators': [1, 1, 1, 1, 1, 0, 1, 1, 1, 1],
 'Description': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'Geographic_Coverage': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'Format': [1, 1, 1, 1, 1, 0, 1, 0, 0, 0],
 'Spatial_Resolution': [1, 1, 1, 1, 1, 0, 1, 1, 1, 0],
 'Temporal_Resolution': [0, 0, 0, 0, 0, 1, 0, 0, 1, 1],
 'Temporal_Extent': [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
 'Latency': [1, 1, 1, 1, 1, 0, 0, 1, 1, 1],
 'Project': [1, 1, 1, 1, 1, 0, 1, 1, 1, 1],
 'Data_Visualization': [1, 1, 1, 1, 1, 1, 1, 0, 1, 0]}

In [158]:
predictions=metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
aspect_metrics = {}
for aspect, preds in predictions.items():
    accuracy = accuracy_score(preds, [1]*len(preds))  # Assuming ground truth is always 1
    precision = precision_score(preds, [1]*len(preds))
    recall = recall_score(preds, [1]*len(preds))
    f1 = f1_score(preds, [1]*len(preds))
    aspect_metrics[aspect] = {'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1-score': f1}

# Print metrics
for aspect, metrics in aspect_metrics.items():
    print(aspect)
    for metric, value in metrics.items():
        print(f"{metric}: {value:.2f}")
    print()

Indicators
Accuracy: 0.90
Precision: 0.90
Recall: 1.00
F1-score: 0.95

Description
Accuracy: 1.00
Precision: 1.00
Recall: 1.00
F1-score: 1.00

Geographic_Coverage
Accuracy: 1.00
Precision: 1.00
Recall: 1.00
F1-score: 1.00

Format
Accuracy: 0.60
Precision: 0.60
Recall: 1.00
F1-score: 0.75

Spatial_Resolution
Accuracy: 0.80
Precision: 0.80
Recall: 1.00
F1-score: 0.89

Temporal_Resolution
Accuracy: 0.30
Precision: 0.30
Recall: 1.00
F1-score: 0.46

Temporal_Extent
Accuracy: 0.50
Precision: 0.50
Recall: 1.00
F1-score: 0.67

Latency
Accuracy: 0.80
Precision: 0.80
Recall: 1.00
F1-score: 0.89

Project
Accuracy: 0.30
Precision: 0.30
Recall: 1.00
F1-score: 0.46

Data_Visualization
Accuracy: 0.80
Precision: 0.80
Recall: 1.00
F1-score: 0.89

